In [3]:
train_data.columns

Index(['label', 'title_sum', 'title_count', 'title_ctr',
       'title_query_dot_similarity', 'title_query_wmd_dis', 'query_word_num',
       'title_word_num', 'is_query_in_title', 'query_len', 'title_len',
       'len_title-query', 'len_query/title', 'q_t_word_match_share',
       'q_t_jaccard', 'q_t_common_words', 'q_t_total_unique_words',
       'q_t_wc_diff', 'q_t_wc_ratio', 'q_t_wc_diff_unique',
       'q_t_wc_ratio_unique', 'title_nunique_prefix', 'Levenshtein_dis',
       'Levenshtein_ratio', 'Levenshtein_jaro', 'Levenshtein_jaro_winkler',
       'longistStr_len', 'prefix_title_pos', 'prefix_title_relative_pos',
       'len_title-query_word', 'len_query/title_word',
       'title_query_leven_rate', 'lcseque_len', 'preifx_detail_title_pos',
       'cityblock_sim', 'norm_similarity', 'cosine_similarity', '0_lsi',
       '1_lsi', '2_lsi', '3_lsi', '4_lsi', '5_lsi', '6_lsi', '7_lsi', '8_lsi',
       '9_lsi', '10_lsi', '11_lsi', '12_lsi', '13_lsi', '14_lsi', '15_lsi',
       '16_lsi'

# 下面是ctr特征生成

### 思路：就是把用来做ctr的那9.5亿，分成16块，每一块分别groupby，会出来16个goupby的结果，
### 然后把title设置成index，用DataFrame.add（DataFrame）这个函数，就可以按索引进行求和。

In [ ]:
import multiprocessing
from time import time
import pandas as pd
import gc
time1=time()
query_title_ctr=pd.DataFrame()
query_ctr=pd.DataFrame()
title_ctr=pd.DataFrame()

# data_path="/home/kesci/input/bytedance/first-round/"
data_path="/home/kesci/input/bytedance/"
data_store_path="first_zzp/"
train_path=data_path+"train_final.csv"


def eva_title_ctr(start_index,single_data_num):
    part_data=pd.read_csv(train_path,header=None,skiprows=start_index,nrows=single_data_num,names=['query_id','query','query_title_id','title','label'],usecols=[1,3,4])
    part_data['label']=part_data['label'].astype(int)
    temp = part_data.groupby(['title'],as_index=False)['label'].agg({'title_sum':'sum','title_count':'count'})
    temp['title_nunique_prefix'] = part_data.groupby(['title'],as_index=False)['query'].agg({'title_nunique_prefix':'nunique'})['title_nunique_prefix']
    return [temp]
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    return alpha, beta


title_ctr_16=[]
pool = multiprocessing.Pool(processes=15)




# for epoch,part_data in enumerate(pd.read_csv(train_path,header=None,chunksize=20000000,names=['query_id','query','query_title_id','title','label'],usecols=[1,3,4])):  
###################说明
###################range里的数，是你要读
time1=time()
for epoch in range(int(15)):    
    single_data_num=int(450000000/15)
    title_ctr_16.append(pool.apply_async(eva_title_ctr, [0+single_data_num*epoch,single_data_num]))
    print(epoch+1)
    
time2=time()
pool.close()
pool.join()
time3=time()
print("groupBy 用时间：",time3-time1)
for index,single_three in enumerate(title_ctr_16):
    time6=time()
    gc.collect()
    single_three=single_three.get()
    title=single_three[0].set_index(['title'])
    if title_ctr.empty:
        title_ctr=title
    else:
        title_ctr=title_ctr.add(title,fill_value=0)
    title_ctr_16[index]=None
    time7=time()
    print("第",index,"轮：",(time7-time6)/60)
title_ctr_16=[]

time4=time()
print((time4-time3)/60)
title_ctr['title_sum']=title_ctr['title_sum'].astype(int)
title_ctr['title_count']=title_ctr['title_count'].astype(int)
title_ctr['title_nunique_prefix']=title_ctr['title_nunique_prefix'].astype(int)
title_ctr['title_ctr']=title_ctr['title_sum']/title_ctr['title_count']
alpha, beta = getBayesSmoothParam(title_ctr['title_ctr'])
title_ctr['title_ctr'] = (title_ctr['title_sum'] + alpha) / (title_ctr['title_count'] + alpha + beta)
te=pd.DataFrame(title_ctr)
title_ctr=title_ctr.reset_index(drop=False)
# title_ctr.to_pickle(data_store_path+"title_ctr.pickle")
title_ctr.to_pickle("/home/kesci/work/first_zzp/ctr/title_ctr_feature_0-4.5.pickle")
time5=time()

print((time5-time4)/60)

# 下面是w2v特征生成
### 此处说明：因为做特征的时间距离比赛结束还剩两天，所以为了防止出错，我没写for循环，而是一个一个cell盯着做的。

In [1]:
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
from gensim.models import Word2Vec
from time import sleep
import multiprocessing
from time import time
import pandas as pd
import gc
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from scipy.stats import skew, kurtosis
import gensim
from gensim.models import Word2Vec
from nltk import word_tokenize
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
import scipy


w2v_model = Word2Vec.load('w2v_model2/w2v_all_data_model.txt')
word_wv = w2v_model.wv


def get_w2v(train_data):
    print("start")
    def get_w2v_feature(word_list):
        w2v_size=100
        word_list[0]=word_list[0].split(' ')
        word_list[1]=word_list[1].split(' ')
        import numpy as np
        wmd_dist = w2v_model.wmdistance(word_list[0], word_list[1])
        if wmd_dist==np.inf:
            wmd_dist=np.nan
        word_vectors_query = np.zeros((len(word_list[0]), w2v_size))
        word_vectors_title = np.zeros((len(word_list[1]), w2v_size))
        for i in range(len(word_list[0])):
            if str(word_list[0][i]) in word_wv.vocab.keys():
                word_vectors_query[i][:] = word_wv[str(word_list[0][i])]
        for i in range(len(word_list[1])):
            if str(word_list[1][i]) in word_wv.vocab.keys():
                word_vectors_title[i][:] = word_wv[str(word_list[1][i])]
        mean_array_query = np.mean(word_vectors_query, axis=0)
        mean_array_title = np.mean(word_vectors_title, axis=0)
        dot_similarity = np.dot(mean_array_title, mean_array_query)
        

        import numpy as np
        return [wmd_dist,dot_similarity]
    train_data = train_data.apply(lambda x : get_w2v_feature(x),axis=1,result_type='expand')
    train_data.columns=['title_query_wmd_dis','title_query_dot_similarity']
    for i in train_data:
        if isinstance(train_data[i],np.float64):
            train_data[i]=train_data[i].astype(np.float32)
        if isinstance(train_data[i],np.int64):
            train_data[i]=train_data[i].astype(np.int16)
    print("w2v feature ended")
    return train_data


pool = multiprocessing.Pool(processes=10)
# funcs=[get_w2v]
# funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/work/first_zzp/train_data0-5.pickle")
# train_data=train_data[['query','title']]
train_data=pd.read_pickle("/home/kesci/work/second_zzp/090-095.pickle")
train_data=train_data[['query','title']]
# test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# train_data=pd.concat([train_data,train_data2,test_data],axis=0)
# train_data=test_data
# del train_data2
# del test_data
# import gc
# gc.collect()

# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))



global_epoch=0
print("start")
fea1='query'
fea2='title'



/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


start
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [2]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=0+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_1.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

0
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [3]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=5000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_2.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

1
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [4]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=10000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_3.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

2
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [5]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=15000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_4.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

3
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [6]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=20000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_5.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

4
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [7]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=25000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_6.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

5
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [8]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=30000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_7.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

6
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [9]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=35000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_8.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

7
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [10]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=40000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_9.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

8
start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


...

start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


start


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended
w2v feature ended


In [11]:
funcs=[get_w2v]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=5000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=45000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/train_w2v_fea/w2v_fea_10.pickle")
all_features_contains_part_train=[]
pool = multiprocessing.Pool(processes=10)

9


# 下面为一些fuzzy、query/title长度、最长公共子串之类的普通特征

In [1]:
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
import Levenshtein
from gensim.models import Word2Vec
from time import sleep
import multiprocessing
from time import time
import pandas as pd
import gc
eooch_tongji=0

###################################
def word_match_share(query,title):
    q1words = {}
    q2words = {}
    for word in query:
        q1words[word] = 1
    for word in title:
        q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R



def jaccard(query,title):
    wic = set(query).intersection(set(title))
    uw = set(query).union(title)
    if len(uw) == 0:
        uw = [1]
    return (len(wic) / len(uw))



def common_words(query,title):
    return len(set(query).intersection(set(title)))



def total_unique_words(query,title):
    return len(set(query).union(title))



def wc_diff(query,title):
    return abs(len(query) - len(title))



def wc_ratio(query,title):
    l1 = len(query)*1.0
    l2 = len(title)
    if l2 == 0:
        return np.nan

    if l1 / l2:
        return l2 / l1

    else:
        return l1 / l2



def wc_diff_unique(query,title):
    return abs(len(set(query)) - len(set(title)))



def wc_ratio_unique(query,title):
    l1 = len(set(query)) * 1.0
    l2 = len(set(title))
    if l2 == 0:
        return np.nan
    if l1 / l2:
        return l2 / l1
    else:
        return l1 / l2




################





def findLongistSubstr(str1, str2):
    '''
    寻找最长公共子串
    '''
    dp = [([0] * len(str2)) for i in range(len(str1))]
    maxlen = maxindex = 0
    for i in range(0, len(str1)):
        for j in range(0, len(str2)):
            if str1[i] == str2[j]:
                if i!=0 and j!=0:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                if i == 0 or j == 0:
                    dp[i][j] = 1
                if dp[i][j] > maxlen:
                    maxlen = dp[i][j]
                    maxindex = i + 1 - maxlen
    return str1[maxindex:maxindex + maxlen]
    
def lcseque_lens(s1, s2): 
     # 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
    m = [ [ 0 for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    # d用来记录转移方向
    d = [ [ None for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    for p1 in range(len(s1)): 
        for p2 in range(len(s2)): 
            if s1[p1] == s2[p2]:            #字符匹配成功，则该位置的值为左上方的值加1
                m[p1+1][p2+1] = m[p1][p2]+1
                d[p1+1][p2+1] = 'ok'          
            elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向
                m[p1+1][p2+1] = m[p1+1][p2] 
                d[p1+1][p2+1] = 'left'          
            else:                           #上值大于左值，则该位置的值为上值，并标记方向up
                m[p1+1][p2+1] = m[p1][p2+1]   
                d[p1+1][p2+1] = 'up'         
    (p1, p2) = (len(s1), len(s2)) 
    s = [] 
    while m[p1][p2]:    #不为None时
        c = d[p1][p2]
        if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个
            s.append(s1[p1-1])
            p1 -= 1
            p2 -= 1 
        if c == 'left':  #根据标记，向左找下一个
            p2 -= 1
        if c == 'up':   #根据标记，向上找下一个
            p1 -= 1
    return len(s)

def get_prefix_loc_in_title(prefix,title):
    if prefix not in title:
        return -1
    lens = len(prefix)
    if prefix == title[:lens]:
        return 0
    elif prefix == title[-lens:]:
        return 1
    else:
        return 2

def get_title_query_levenshtein_distance(df):
    chinese_dict='事丸书万丑丕丽举其冀匈匙冯冲决冻况冷冶净凋凉凌准凑减凛卢卦厂历厅压厌厕厘厝厩厢厨厥切券剪劈刊刚列刘刓刎刖刨刬刭刺到刮刿剀刻刳刷剉剐剑剟剧剖剡剔剜副割剩剿剽党亏匜匡匣匽匪匿匮队阡邦防阶阱阬那阮邪阳阿陂陈邸阽附际邻陇陀邺阻阼陌陔郊郎陋限郁郑陛除陡郛郡郤险院陨部陲都郭陵陪陴陶陬隍隆隋随隈隅隘隔鄙鄘障隧隤隰酂兰关兹养凤却冈罔冕办劝务动助劲励努劬劭劾势勃勉勋勍勘勒勔募勤写冢击凿余亿仇仅仍代伋仞仪仗伧伥仿份伎价伉伦仳任伤似伟优伛伯伺但低佃伽估何伶你伾佗佚佣佑住侧侪侈佽供佳侥佼例侣侔侬侨使侍佻侠佯依侑侦俦促俄俘俭俊俚俪俩俜俟俏俗侮修俨俑倍俾倡俶倒俸候健借俱倨倦倔倥倪俳倩倏倭倚债值偿偾假傀偻偶偏停偷偎偕偃傲傍傧储傣傅傩傥催像儆僦僚僧僮僖僵僻儇儒千博产亩亲亵亶订讣讥认讧记讦讫让讱讪训议讹访讽讳讲诀论讴讼讻许讶诐词诋评诎识诉译诈诊证诅诧诚诞该诟诡话诨诘诓诔诠诗试详诩询诣诤诛诶诰诲诫诳诮说诵诱语谄调读诽课谅诺请谁谂谇诿谀诸谆诼诹谈谙谗谛谍谔谎谏谋谓谐谖谑谚谒谕谠谧谟谦谥谢谣谨谩谬谪谰谱谭谮谴谳谶廷发变叛叙艾节芒芃芋芭苍芳芙芥芦芡芮苏苇芜芽芸芷茇苞范苟苦茄茎苴苛苓茏茅茂茉苹苒苫苕苔英茔苑荜草茬茨荒荟荤荚荐茭荆荦茫茗茜荃荛荏茸荣茹荪荑荀药茵荫莞荷获莱莅莲莎荼莺莹莠菜萃菲菡菅菁菊菌萝萌萋菘萎萧营萦著菑菹葆葱蒂董葛葫蒋葵落葩葺葸萱蒉蒿蒺蓟蓝蒙蓦蓬蒲蓐蒜蓄蒸蔽蔻蔓蓰箫蔗蕉蕤蕊蔬蕴薄薨蕾薮藐藓薰藩藤藻蘖彻彷彼徂径征待很徊律徇徒徐徜徘徙循徯徭德徽巡边辽达过迈迄迁迆迂迟返还迒近违迕迓迎远这迨迪迭迩迥迫述迢逅迷逆适逃选逊逋逞递逗逢逦逑逡逝速通透途逍造逴逮逵逻逶逼遍遄道遁遏遑遒遂遗遐逾遇遨遘遛遣遥遭遮遴遵避遽邂邀邅邈邃导寺寿将失头夸奅奖奎奕奢奡奥干年巧巩弘弛张弧弥弩弭弯弹强弈弊广庇床庐庑序底店废府庙庞庖度庭庠座廊庾廋廒廓廉当录彝币布帏帐帛帘帔帖帜帙帮帧常帷帻幅帽幂幄幕幔幛幢幡叱叮句叫叶台叹召只吃吓吐吸吁吵呈吨含吼吝呕吮听吞吻吾呀吟呵咄呷咖咍呼咀呶咛咆呻味咏哀咤哈咳哄哗哪咷哇响哑咽咬哉咱咫咨啊唉哺唇哽哼唤唧唠哨唆唐哮唁哲唱啜啖啃售啴唾唯啸啄喳啻喘喋喊喝喙嗟喈喟喃喷啼嗢喧喑喁喻嗞嗷嗔嗤嗝嗑嗜嗣嗅嘎嘉嘘嗾嗽嘤嘲嘬嘿噎嘶嘻嘱噫噬嚎嚼囊驰驮驯驴驱驲骀驸驾驹驽驶驷驼驿驻驺骇骄骆骂骈骁骋骏验骖骐骑骗骚骛骘骜骞骤问闳闷闵闱闸阀阁闺阂闿闾闽闼闻阃阅阐阊阍阎阔阑阕阖阙阗宄宇宅宏完宝宠宛宙客宣宥宾宸宽宵宴寄寂密宿富寐寓寝察寨寮寰奶妈她妄妆妣妒妨妓妊姒妩妍妖妪姊姑姐姆妹妮始姹姜娇姣姱姥'
    step=0
    def dis(word_string):
        query_len = len(word_string['query']) 
        title_len = len(word_string['title'])
        len_title_jian_query = title_len - query_len
        len_query_div_title = query_len / title_len
        
        word_set=set(word_string['query'].split()+word_string['title'].split())
        word_dict={word:chinese_dict[index] for index,word in enumerate(word_set)}
        query=''
        title=''
        for i in word_string['query'].split():
            query+=word_dict[i]
        for i in word_string['title'].split():
            title+=word_dict[i]
        query_word_num=len(query)
        title_word_num=len(title)
        len_title_jian_query_word=title_word_num-query_word_num
        len_query_div_title_word=query_word_num/title_word_num
        is_query_in_title=0
        if query in title:
            is_query_in_title=1
        Levenshtein_dis=Levenshtein.distance(query, title)
        Levenshtein_ratio=Levenshtein.ratio(query, title)
        Levenshtein_jaro=Levenshtein.jaro(query, title)
        Levenshtein_jaro_winkler=Levenshtein.jaro_winkler(query, title)
        title_query_leven_rate=(Levenshtein_dis / (len(title) + 3))
        longistStr_len=len(findLongistSubstr(word_string['query'],word_string['title']))
        prefix_title_pos=title.find(query)
        prefix_title_relative_pos=prefix_title_pos/len(title)
        
        list_query=list(query)
        list_title=list(title)        
        word_match_share2=word_match_share(list_query,list_title)
        jaccard2=jaccard(list_query,list_title)
        common_words2=common_words(list_query,list_title)
        total_unique_words2=total_unique_words(list_query,list_title)
        wc_diff2=wc_diff(list_query,list_title)
        wc_ratio2=wc_ratio(list_query,list_title)
        wc_diff_unique2=wc_diff_unique(list_query,list_title)
        wc_ratio_unique2=wc_ratio_unique(list_query,list_title)
        lcseque_len2=lcseque_lens(list_query,list_title)
        preifx_detail_title_pos2=get_prefix_loc_in_title(list_query,list_title)
        
        return [query_len,title_len,len_title_jian_query,len_query_div_title,
            query_word_num,title_word_num,len_title_jian_query_word,is_query_in_title,
            len_query_div_title_word,Levenshtein_dis,Levenshtein_ratio,
            Levenshtein_jaro,Levenshtein_jaro_winkler,title_query_leven_rate,
            longistStr_len,prefix_title_pos,prefix_title_relative_pos,
            word_match_share2,jaccard2,common_words2,total_unique_words2,wc_diff2,wc_ratio2,wc_diff_unique2,wc_ratio_unique2,
            lcseque_len2,preifx_detail_title_pos2]
    print("distance ended")
    temp_df=df[['query','title']].apply(dis,axis=1,result_type='expand')
    temp_df.columns=['query_len','title_len','len_title-query','len_query/title',
    'query_word_num','title_word_num','len_title-query_word','is_query_in_title',
    'len_query/title_word','Levenshtein_dis','Levenshtein_ratio',
    'Levenshtein_jaro','Levenshtein_jaro_winkler','title_query_leven_rate',
    'longistStr_len','prefix_title_pos','prefix_title_relative_pos',
     'q_t_word_match_share','q_t_jaccard','q_t_common_words','q_t_total_unique_words','q_t_wc_diff','q_t_wc_ratio'
     ,'q_t_wc_diff_unique','q_t_wc_ratio_unique','lcseque_len','preifx_detail_title_pos']
    return temp_df




In [5]:
for i in range(20):
    train_data[i*5000000:(i+1)*5000000].to_pickle("/home/kesci/test_finall_all_parts/part_"+str(i)+".pickle")

In [2]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
train_data=pd.read_pickle("/home/kesci/test_final_fea/test_data.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_1.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [3]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=10000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_2.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [4]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=20000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_3.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [5]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=30000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_4.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [6]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=40000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_5.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [7]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=50000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_6.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [8]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=60000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_7.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [9]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=70000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_8.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [10]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=80000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_9.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


In [11]:
pool = multiprocessing.Pool(processes=10)
# # funcs=[get_w2v]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance]
# # funcs=[get_query_word_num,get_title_word_num,get_is_query_in_title_feature,get_string_len_feature,get_title_query_levenshtein_distance,get_w2v]
all_features_contains_part_train=[]
# train_data=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
# # test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_data.pickle")
# # test_data[['query']]=test_data[['query']].applymap(lambda x:x.replace('\t',''))
# train_data[['query']]=train_data[['query']].applymap(lambda x:x.replace('\t',''))
# # test_data[['title']]=test_data[['title']].applymap(lambda x:x.replace('\t',''))
# train_data[['title']]=train_data[['title']].applymap(lambda x:x.replace('\t',''))
global_epoch=0
print("start")


fea1='query'
fea2='title'
funcs=[get_title_query_levenshtein_distance]
for func in funcs:
    print(global_epoch)
    global_epoch+=1
    all_train_data_size=10000000
    cpu_size=10
    single_part_of_train_size=int(all_train_data_size/cpu_size)
    one_feature=[]
    for part_id in range(cpu_size):
        start=90000000+part_id*single_part_of_train_size
        end=start+single_part_of_train_size
        single_part=train_data[start:end]
        one_feature.append(pool.apply_async(func, [single_part]))
    all_features_contains_part_train.append(one_feature)

pool.close()
pool.join()
for index,i in enumerate(all_features_contains_part_train):
    single_contains_part=[]
    for j in i:
        single_contains_part.append(j.get())
    all_features_contains_part_train[index]=single_contains_part
all_features_contains_part_train=all_features_contains_part_train[0]

all_features_contains_part_train=pd.concat(all_features_contains_part_train)
all_features_contains_part_train.to_pickle("/home/kesci/blending/char_fea_10.pickle")

start
0
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended
distance ended


# lsi特征（tsvd降维特征）

## 下面是做词典

In [ ]:
from time import time 
import pandas as pd
import warnings

# query=pd.read_pickle("/home/kesci/work/query_all.pickle")
# title=pd.read_pickle("/home/kesci/work/title_all.pickle")

time1=time()
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
from gensim import corpora,similarities,models
# test_data_all=pd.read_pickle("query_all.pickle")
class mycorpus(object):
    def __iter__(self):
        for index,doc in enumerate(test_data_all['query']):
            if index%5000000==0:
                print(index)
            yield doc.split(' ')
corp = mycorpus()
# dictionary.add_documents(corp)
dictionary=corpora.Dictionary(corp)
time2=time()
print(time2-time1)

## 下面是做tfidf

In [ ]:
from time import time 
import warnings
time1=time()
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
from gensim import corpora,similarities,models
import pandas as pd
# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")

print(111523155656)
class MyCorpus(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(test_data_all['query']):
            if index%5000000==0:
                print(index)
            yield dictionary.doc2bow(doc.split(' '))
corpus = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
tfidf_model = models.TfidfModel(corpus, id2word=dictionary)
# corpus_tfidf = tfidf_model[corpus]

## 训练lsi模型（即tsvd模型）

In [ ]:
from time import time 
import warnings
time1=time()
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
from gensim import corpora,similarities,models
import pandas as pd
# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")
corpus_tfidf = tfidf_model[corpus2]
print(111523155656)
class MyCorpus(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(corpus_tfidf):
            if index%2000000==0:
                print(index)
            yield doc
corpus3 = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
lsi_model = models.LsiModel(corpus3, id2word=dictionary,chunksize=10000000,num_topics=20)
lsi_model.save("lsi_model/lsi_all_query.lsi")
time2=time()
print((time2-time1)/60)
# corpus_tfidf = tfidf_model[corpus]

## 使用lsi模型

### 此处说明：将tfidf值传入lsi后，lsi返回的是一个迭代器，每次迭代返回一个list，需要用pd.DataFrame来将list转为DataFrame。 此处用了多进程，分批将list转为DataFrame。

In [ ]:
from gensim import models
lsi_model=models.LsiModel.load("/home/kesci/work/lsi_model/lsi_all_query.lsi")
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

import multiprocessing
import pandas as pd
# lsi_list=data1
# del data1
# import gc
# gc.collect()

lsi_columns=[str(i)+'_lsi' for i in range(20)]
def to_df(start,end,train_data):
    print("started")
    lsi_list=[dictionary.doc2bow(text.split(' ')) for text in train_data['query'][start:end]]
    lsi_list=lsi_model[tfidf_model[lsi_list]]
    print("doc2bow")
    temp_df=pd.DataFrame(list(lsi_list),columns=lsi_columns)
    print("apply_map")
    temp_df=temp_df.applymap(lambda x:x[1] if x!=None else np.nan)
    print("ended")
    return temp_df
pool = multiprocessing.Pool(processes=16)
single_data_num=int(5000000/16)
all_data=[]

In [ ]:
for part_id in range(20):
    pool = multiprocessing.Pool(processes=16)
    all_data=[]
    train_data=pd.read_pickle("/home/kesci/test_finall_all_parts/part_"+str(part_id)+".pickle")
    train_data=train_data.reset_index(drop=True)
    print("start")
    for i in range(16):
        start=0+i*single_data_num
        end=start+single_data_num
        all_data.append(pool.apply_async(to_df, [start,end,train_data]))
    pool.close()
    pool.join()
    all_data=[single_part.get() for single_part in all_data]
    all_data=pd.concat(all_data)
    all_data=all_data.reset_index(drop=True)
    # all_data_parts=[]
    # all_data_parts.append(all_data)
    all_data.to_pickle("/home/kesci/test_final_lsi/lsi_part_"+str((part_id+1))+".pickle")